In [ ]:
import time
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator                           

In [ ]:
train_dir = 'dataset/train'
val_dir = 'dataset/validation'
test_dir ='dataset/test'

IMG_SIZE = 299
BATCH = 16

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH,
    class_mode='sparse',  # Assumes one-hot encoded labels
    shuffle=True,
)

# Validation data generator
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH,
    class_mode='spars',  # Assumes one-hot encoded labels
    shuffle=True,
)

# Testing data generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH,
    class_mode='spars',  # Assumes one-hot encoded labels
    shuffle=True,
)

classes = sorted(train_generator.class_indices.keys())
print(classes)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3

# Define the base model with pre-trained weights
base_model = InceptionV3(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')

# Freeze the base model with 50% of its layers
for layer in base_model.layers[:int(len(base_model.layers) * 0.5)]:
    layer.trainable = False

# Add custom classification layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x) 
predictions = Dense(2, activation='softmax')(x)

# Compile the model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='spars',
              metrics=['accuracy'])

In [ ]:
VAL_ACC_THRESHOLD = 0.90
VAL_LOSS_THRESHOLD = 0.01

class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            val_acc = logs.get('val_accuracy')
            val_loss = logs.get('val_loss')
            if val_acc is not None and val_loss is not None:
                if val_acc > VAL_ACC_THRESHOLD and val_loss < VAL_LOSS_THRESHOLD:
                    print(f"\nVal_accuracy lebih dari {VAL_ACC_THRESHOLD*100}% dan val_loss kurang dari {VAL_LOSS_THRESHOLD}.")
                    self.model.stop_training = True

callbacks = CustomCallback()

In [ ]:
# Record the start time
start_time = time.time()

# Model training
num_epochs = 100
history = model.fit(train_generator, 
                    epochs=num_epochs, 
                    validation_data=val_generator,
                    callbacks=[callbacks])

# Record the end time
end_time = time.time()
training_time = end_time - start_time
print(f'Training time: {training_time} seconds')

# Save the model
model.save('models/InceptionV3_0001_50.h5')

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

# Get the true labels and predictions
true_labels = test_generator.classes
predictions = model.predict(test_generator).argmax(axis=1)

# Generate the confusion matrix
cm = confusion_matrix(true_labels, predictions)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()

# Add labels to the axes
classes = list(test_generator.class_indices.keys())
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

# Add the numbers on the squares
thresh = cm.max() / 2.
for i, j in np.ndindex(cm.shape):
    plt.text(j, i, f"{cm[i, j]}", horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

# Print classification report
print("Classification Report:")
print(classification_report(true_labels, predictions, target_names=classes))